# 3.2: Inspecting the `data` directory

`run_prepare_data.sh` will generate a new directory,`data`, that contains many of the files required for the `ASR` pipeline.  We will inspect its contents below.

**Note:** The official `kaldi` documentation has a more detailed explanation of these files [here](http://kaldi-asr.org/doc/data_prep.html).  Just beware that **some** files explained there are not relevant to our pipeline.

In [ ]:
ls data

## `data/kaldi_config_args.json`

This file is simply a *copy* of the `kaldi_config.json` used in the running of `run_prepare_data.sh`.  Each major step of the ASR pipeline will copy the relevant portion of the `kaldi_config.json` file to the relevant location so that we can keep track of what we used for the experiment.

In [ ]:
cat data/kaldi_config_args.json

## `data/{train|test}_dir`

These directories contain four files for each of the subsets, `train` and `test` (assuming you set up a configuration with both when you ran `run_prepare_data.sh`, which we did).

In [ ]:
ls data/train_dir

### `wav.scp`

This file maps each `audio basename` to its full path.  It takes the format:

```
[audio-basename] [full/path/to/audio]
[audio-basename] [full/path/to/audio]
[audio-basename] [full/path/to/audio]
```

In [ ]:
head -n5 data/train_dir/wav.scp

**Note:** `kaldi` allows the second argument of this file to be a `piped` command.  See this example from the official documentation:

```
s5# head -3 data/train/wav.scp
sw02001-A /home/dpovey/kaldi-trunk/tools/sph2pipe_v2.5/sph2pipe -f wav -p -c 1 /export/corpora3/LDC/LDC97S62/swb1/sw02001.sph |
sw02001-B /home/dpovey/kaldi-trunk/tools/sph2pipe_v2.5/sph2pipe -f wav -p -c 2 /export/corpora3/LDC/LDC97S62/swb1/sw02001.sph |
```

In this case `/home/dpovey/kaldi-trunk/tools/sph2pipe_v2.5/sph2pipe -f wav -p -c 1 /export/corpora3/LDC/LDC97S62/swb1/sw02001.sph` is a command that will convert a `.sph` file to `wav`.  The final character, `|`, acts as if it were `piping` the output to a file location.  

In our case, we explicitly converted the audio as a preprocessing step, but one can easily follow this approach if it's important that the original audio file not be modified.

### `text`

This file is a subset of the `transcripts` file supplied to `run_prepare_data.sh` that contains **only** the transcripts of the utterances in that particular subset.

In [ ]:
head -n5 data/train_dir/text

### `utt2spk`

This file maps each utterance to a speaker.  Having speaker information becomes important later when we try to make certain speaker-level adaptations to the model (in an attempt to generalize the characteristics of a particular speaker). 

The file takes the following format:

```
[utterance-id] [speaker-id]
[utterance-id] [speaker-id]
[utterance-id] [speaker-id]
```

In [ ]:
head -n5 data/train_dir/utt2spk

**Note:** If we weren't to have any information about speakers in our dataset, we could simply make the `speaker-id` the same as the `utterance-id`, which would result in `n` unique speakers (where `n` is the number of utterances in the subset).

And so you can see from the file, this `kaldi` pipeline works under the assumption that we do **not** have any information about speakers.  This is not particularly true in the case of the `librispeech` dataset, however.

One of the files created by an intermediate step during `1:1 Downloading...` created an `utt2spk` file for each of the original `librispeech` subsets.  They are stored in `raw_data/Librispeech/[subset]_data`.

In [ ]:
head -n5 raw_data/LibriSpeech/dev-clean_data/utt2spk

Here you can see that the `utterance-id` convention is such that the last portion `000*` refers to a segment of a larger audio directory, all spoken by the same person.

**Note:** We will stick with our current `utt2spk` for the sake of continuity in our pipeline, but at a later date, we can revisit this and see how performance changes (improves?) when using an `utt2spk` file that takes speaker information into account. 

### `spk2utt`

This file is simply a reverse mapping of `utt2spk`.  Instead of mapping an uttreance to a speaker, this file maps a speaker to all of her or his spoken utterances.  It thus takes the format:

```
[speaker-id] [utterance-id_1] [utterance-id_2] [utterance-id_3] ... [utterance-id_n]
[speaker-id] [utterance-id_1] [utterance-id_2] [utterance-id_3] ... [utterance-id_n]
[speaker-id] [utterance-id_1] [utterance-id_2] [utterance-id_3] ... [utterance-id_n]
```

In our case, this will be identicial to `utt2spk` since we are not taking speaker information into account (and we thus have a unique speaker for each utterance).

In [ ]:
head -n5 data/train_dir/spk2utt

But looking at the `spk2utt` in `raw_data/LibriSpeech/[subset]`, we can see what a `spk2utt` would like that **does** take speaker information into account.

In [ ]:
head -n2 raw_data/LibriSpeech/dev-clean_data/spk2utt

### `segments` (optional)

This file does **not** appear in our pipeline when using `librispeech` data, but if were accessing audio that was **not** already segmented, we would have supplied a master `segments` file to `run_prepare_data.sh`, which would have generated a `{train|test}`-specific `segments` file here.  It would take the following format:

```
[audio-basename] [utterance-id] [utterance-start] [utterance-stop]
[audio-basename] [utterance-id] [utterance-start] [utterance-stop]
[audio-basename] [utterance-id] [utterance-start] [utterance-stop]
```

## `data/local`

This directory is an intermediate (essentially, `temp`) directory used for housing files as they are manipulated and/or built for later use.  All the **important** files will appear in another subdirectory of `data`, so we won't spend too much time on the items here.

### `waves.{train|test}`

These files are simply a list of the audio files that belong to the `train` and/or `test` subsets.

In [ ]:
head -n5 data/local/waves.train
echo ...
head -n5 data/local/waves.test

### `lm_tg.arpa`

This is a modified version of the `language model` that you supplied as an argument to `run_prepare_data.sh` with any `n-gram` containing `<UNK>` removed.  This will ensure that our model will **never** predict `<UNK>` when decoding.

In [ ]:
diff data/local/lm_tg.arpa raw_data/3-gram.pruned.3e-7.arpa | head -n10

### `data/local/dict`

This directory contains files pertaining to the `lexicon`.

In [ ]:
ls data/local/dict

#### `lexicon.txt`

This is just a local copy of the `lexicon` you supplied to `run_prepare_data.sh`

In [ ]:
diff data/local/dict/lexicon.txt raw_data/librispeech-lexicon.txt

**Note:** `kaldi` will do this alot: it will **copy** files to a `local` location and then call those files from that **local** location (as opposed to their original locations.  This is inefficient from a disk space perspective in my opinion, but it would be a tremendous amount of work to parameterize all the scripts to take a location.  And disk space is cheap...So we will suffer it.

#### `lexiconp.txt`

This is a form of the `lexicon` with an additional value for each word: the probability of that pronunciation.

In [ ]:
cat data/local/dict/lexiconp.txt | grep INDIRECTLY

This allows for you to provide a `lexicon` that not only provides alternative pronunciations, but weights them according to their likelihood.  In our case, however, we don't have data to support the setting of those values, so all pronunciations are equally weighted at `1.0` (yes, it probably should be `.5` and `.5`, but `kaldi` is OK with `1.0` all the different pronunciations of a given word).

#### `{non}silence_phones.txt` and `optional_silence.txt`

`silence_phones.txt` and `nonsilence_phones.txt` simply separate the `phones` we supplied to `run_data_prepare.sh` into those that refer to `silence` and those that don't.  In our case, the only `silence` phone is `SIL`.

In [ ]:
cat data/local/dict/silence_phones.txt

`optional_silence.txt` contains the value for a `phone` we will use to identify the `silence` between words.  The official `kaldi` documentation linked above doesn't go into much detail as to why this is used.

In [ ]:
cat data/local/dict/optional_silence.txt

### `data/local/lang`

This directory is truly only a `temp` `directory` used in the building of `data/lang`.  So we will skip investigating this directory, and move on to `data/lang`. 

## `data/lang`

This directory will contain all the files needed to utilize the `language model` (how `kaldi` accesses the `ARPA`-format `language model` during decoding will be discussed later).

### `words.txt`

This file is a mapping of each word in our `lexicon` to an index.

**Note:** `kaldi` does this often.  You will see many examples of files like this that map something to an index for efficiency.

In [ ]:
head -n5 data/lang/words.txt
echo ...
tail -n5 data/lang/words.txt

You'll notice a few added "words" to this list (more on each of these in later weeks):

 - `<eps>`: a faux-word used to model the "space" between words in a later step
 - `#0`: a faux-word used to allow our `finite state transducer (FST)` to function properly
 - `<s>`: an faux-word used to model the "start of an utterance" in a later step
 - `</s>`: an faux-word used to model the "end of an utterance" in a later step

### `phones.txt`

This file is a mapping of each phone (from the list that we supplied to `run_prepare_data.sh`) to an index.

In this case, there are two additional differences between this file and the one we supplied to `run_prepare_data.sh`:

 - an additional `<eps>` phone is added (more about the role this plays later)
 - each phone is converted to `BIES` notation, which identifies where the phone occurs in the word 
   - `B`eginning
   - `I`nside
   - `E`nd
   - `S`olo (the word is made up of only this phone)

In [ ]:
head data/lang/phones.txt

### `{arpa}_oov.txt`

These two files identify any words in the `lexicon` that do **not** appear in our `language model` (`OOV` = out of vocabulary).

**Note:** This will hopefully become evident later, but this would cause a significant problem for our `ASR` model, and so we took steps during the preparation stage to identify any such words.

In [ ]:
cat data/lang/arpa_oov.txt
echo ...
cat data/lang/oov.txt

In our case, since our `language model` and `lexicon` were both built directly from the same source, it shouldn't be surprising to see an "empty" list.  But you can imagine a situation where your `language model` would be built from a very large corpus, and while there are ways to automatically generate (approximate) pronunciations of words (see [here](https://www-i6.informatik.rwth-aachen.de/web/Software/g2p.html) for one such program), it may not be an efficient step in the data preparation to worry about ensuring all words are present in both files.

In a later step, we will automatically **remove** any such `OOV` words from the final `FST`.

### `oov.int`

This file is a representation of `oov.txt` using indices instead of strings.

**Note:** `kaldi` will do this a lot.  And the filetypes (`.txt` v. `int`) will be an indication of which form to expect.  

In [ ]:
cat data/lang/oov.int

### `topo`

This file specifies architecture of the `Hidden Markov Model`s (`HMM`s) we will use in the building of our acoustic model.  This will be discussed in more detail later.

In [ ]:
cat data/lang/topo

For now, we can see two different `HMM` architectures, one for phones `1,2,3,4,5`, and one for all other phones.  Revisiting `data/lang/phones.txt` we can see that phones `1-5` are `silence phones`.

In [ ]:
cat data/lang/phones.txt | head -n 6

### `L.fst` and `L_disambig.fst`

These two files are the `finite state transducer` (`FST`) representations of our language model.  `L.fst` is the `FST` representation of our language model.  Without going into any of the details about this, `L_disambig.fst` is a modified version of the `L.fst` that is necessary for efficient usage of the `FST`.  If you're interested you can read more [here](http://kaldi-asr.org/doc/graph.html#graph_disambig).

But for now you can see below that `L_disambig.fst` is bigger than `L.fst`.  It has more `states` and `arcs`.

In [ ]:
diff <(/home/kaldi/tools/openfst/bin/fstinfo data/lang/L.fst) <(/home/kaldi/tools/openfst/bin/fstinfo data/lang/L_disambig.fst)

**Note:** using `diff <(...) <(...)` allows us to look at the difference of two outputs to `STDOUT`.

### `data/lang/phones`

This directory contains more files pertaining to our phones set.

#### `silence.{txt|int|csl}, nonsilence.{txt|int|csl}, and optional_silence.{txt|int|csl}`

We already saw the `.txt` versions of these files in `data/local/dict`. And you will recall that `.int` files are simply an alternative form of `.txt` files using indices.  `.csl` (colon-separated list) files are yet another version of `.txt` files:

In [ ]:
cat data/lang/phones/silence.txt

In [ ]:
cat data/lang/phones/silence.csl

#### `word_boundary.{txt|int}`

These files are mappings of each phone in BIES notation to its location in the word. The kaldi documentation explains why such files are important very clearly here:

```
This is the same information as is in the suffixes of the phones (_B and so on), but we don't like to hardcode this in the text form of the phones– for one thing, Kaldi executables never see the text form of the phones, but only an integerized form.
```

Being able to identify this information will help, as the filenames imply, when trying to recover word boundaries from a sequence of sounds.

In [ ]:
head data/lang/phones/word_boundary.txt

#### `align_lexicon.{txt|int}`

These files are created whenever position-dependent (e.g. `BIES`) phones are used, which is true in our case. And these files simply update the lexicon to include the position-dependent forms of each phone.

Compare `data/lang/phones/align_lexicon.txt` with our original lexicon.

In [ ]:
diff <(tail data/lang/phones/align_lexicon.txt) <(tail raw_data/librispeech-lexicon.txt)

#### `context_indep.{txt|int|csl}`

These files contain a list of the phones that we will not model with context (*i.e.* we don't need to pay attention to what phones come before or after them). You'll notice that the phones considered "context-independent" are the ones that do not refer to "real phones". In our case, they are the exact same phones as the "silence" phones.

In [ ]:
cat data/lang/phones/context_indep.txt

#### `disambig.{txt|int|csl}`

If you recall, we saw two different forms of the lexicon represented as `FST`s, one of which was called `L_disambig.fst` which contained extra `arcs` and `states`.  These files identify those extra symbols, called `disambiguation symbols`.

In [ ]:
cat data/lang/phones/disambig.txt

#### `sets.{txt|int}`

These files group all of our `BIES`-notated phones into clusters that all refer to the same phone. In other words, we put the `B_`, `_I`, `_E`, and `_S` forms of each phone onto one line together. This will become important when we try to model our language using `HMM`s in later steps of the pipeline.

In [ ]:
head data/lang/phones/sets.txt

#### `extra_questions.{txt|int}`

These files contain a different way of grouping phones than `sets.{txt|int}` did it.

**Note:** These files are called "questions" because they will be the "questions" we ask to help decide how to split the data during the building of our decision tree to model phonetic context early in the acoustic modeling stage.

You'll notice these files group all the `_B` non-silence phones together, all the `_I` non-silence phones together, etc. They then do the same thing for the `silence phones`, which, consist only of `SIL` in our case.

In [ ]:
cat data/lang/phones/extra_questions.txt

#### `roots.{txt|int}`

These files will be helpful when we build the decision tree to model phonetic context.

We will revisit this decision tree later, but here's how the kaldi documentation explains this file:

```
The significance of having a number of phones on a single line, for example SIL SIL_B SIL_E SIL_I SIL_S, is that all of these phones have a single "shared root" in the decision tree, so states may be shared between those phones. For stress and tone-dependent systems, typically all the stress or tone-dependent versions of a particular phone will appear on the same line. In addition, all three states of a HMM (or all five states, for silences) share the root, and the decision-tree building process gets to ask about the state. This sharing of the decision-tree root between the HMM-states is what we mean by "shared" in the roots file.
```

In [ ]:
head -n5 data/lang/phones/roots.txt